In [1]:
import cv2
import numpy as np 
import pandas as pd 
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pre
from glob import glob
from sklearn.utils.class_weight import compute_class_weight
os.environ["CUDA_VISIBLE_DEVICES"] = '1,2'


2023-07-02 17:41:30.525057: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-02 17:41:30.648248: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-02 17:41:31.206697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/usr/local/cuda-11.6/lib64:/home/gil/anaconda3/envs/LeeYS/lib/
2023-07-02 17:41:31.206765: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library

In [2]:
Train_img_list=pd.read_csv('../../data/preprocessing_dataset/Train_data_set.csv')['file_path'].to_list()
Train_label_list=pd.read_csv('../../data/preprocessing_dataset/Train_data_set.csv')['standard_frame'].to_list()
Test_img_list=pd.read_csv('../../data/preprocessing_dataset/Test_data_set.csv')['file_path'].to_list()
Test_label_list=pd.read_csv('../../data/preprocessing_dataset/Test_data_set.csv')['standard_frame'].to_list()

In [3]:
Train_img_list=glob('../../data/6_Train_styleGAN/img/Normal/*.jpg')
Train_img_list1=glob('../../data/6_Train_styleGAN/pred/*.jpg')
Train_img_list2=glob('../../data/6_Train_styleGAN/img/Abnormal/*.jpg')
Train_img_list1.extend(Train_img_list2)
Train_label_list=np.zeros(len(Train_img_list)).tolist()
Train_label_list1=np.ones(len(Train_img_list1)).tolist()
Train_img_list.extend(Train_img_list1)
Train_label_list.extend(Train_label_list1)

In [4]:
size=256
x_train = np.zeros((len(Train_img_list),size,size,3))
y_train = np.zeros((len(Train_label_list),1))
for i in range(len(Train_img_list)):
    x_train[i] =np.array(Image.open(Train_img_list[i]).resize((size,size)))
x_train=x_train/255
y_train=np.array(Train_label_list)

x_test = np.zeros((len(Test_img_list),size,size,3))
y_test = np.zeros((len(Test_label_list),1))
for i in range(len(Test_img_list)):
    x_test[i] =np.array(Image.open('../../data/preprocessing_dataset/'+Test_img_list[i]).resize((size,size)))
x_test=x_test/255
y_test=np.array(Test_label_list)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [6]:

checkpoint_filepath = "../../model/resnet50_style_preprocessing_checkpoints.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only= True
)
class_weight_ratio=compute_class_weight(class_weight = "balanced" , 
                     classes=np.unique(y_train), 
                     y = y_train)
class_weight = {0:class_weight_ratio[0],1:class_weight_ratio[1]}
mirrored_strategy = tf.distribute.MirroredStrategy(devices=['/GPU:0','/GPU:1'])
with mirrored_strategy.scope(): 
    input_t=K.Input(shape=(size,size, 3))
    input_tensor = layers.experimental.preprocessing.Resizing(size, size, interpolation="bilinear", input_shape=x_train.shape[1:])(input_t)
    ResNet=ResNet50(include_top=True,weights='imagenet',input_tensor=input_tensor)
    model = K.models.Sequential()
    model.add(ResNet)
    model.add(tf.keras.layers.Dropout(.2, input_shape=(64,)))
    model.add(K.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
    model.add(K.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=K.optimizers.Adam(lr=2e-6),
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=["accuracy"])
    histo=model.fit(
        x_train,y_train,
        validation_data=(x_val,y_val),
        epochs=500,
        callbacks=[model_checkpoint_callback],
        batch_size=64,shuffle=True,
        class_weight=class_weight
    )
model.save('../../model/ResNet50_style_preprocessing256.h5')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


Epoch 1/500
INFO:tensorflow:batch_all_reduce: 218 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 218 all-reduces with algorithm = nccl, num_packs = 1
135/135 [==============================] - 40s 190ms/step - loss: 0.6876 - accuracy: 0.6246 - val_loss: 0.6946 - val_accuracy: 0.2427
Epoch 2/500
135/135 [==============================] - 21s 155ms/step - loss: 0.6630 - accuracy: 0.8907 - val_loss: 0.6938 - val_accuracy: 0.3004
Epoch 3/500
135/135 [==============================] - 21s 152ms/step - loss: 0.6570 - accuracy: 0.9255 - val_loss: 0.6925 - val_accuracy: 0.7652
Epoch 4/500
135/135 [==============================] - 21s 153ms/step - loss: 0.6525 - accuracy: 0.9291 - val_loss: 0.6916 - val_accuracy: 0.7456
Epoch 5/500
135/135 [==============================] - 21s 155ms/step - loss: 0.6467 - accuracy: 0.9330 - val_loss: 0.6907 - val_accuracy: 0.7701
Epoch 6/500
135/135 [==============================] - 21s 159ms/step - loss: 0.6421 - accuracy:

In [ ]:
np.array(Train_label_list).min()